In [1]:
# %%
import os
os.environ["RPY2_CFFI_MODE"] = "ABI"
import rpy2.robjects as ro
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from rpy2.rinterface_lib.callbacks import logger as rpy2_logger
rpy2_logger.setLevel("ERROR")   # 只显示错误，屏蔽 message 和 warning
rpy2_logger.propagate = False   # 阻止继续传给 root logger

import sys
sys.path.append("../../")

# from scSurvival.scsurvival import scSurvival, scSurvivalRun, PredictIndSample
from scSurvival_beta import scSurvivalRun, PredictIndSample
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os 
os.environ['KMP_DUPLICATE_LIB_OK']='True'

from tqdm import tqdm, trange
import scanpy as sc
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, NMF
from sklearn.cluster import KMeans

from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
import io
import contextlib
f = io.StringIO()
from lifelines.utils import concordance_index
from scipy.stats import percentileofscore
from utils import *
import importlib

from trainditional_cox_cv import nested_cv_coxnet, cv_rsf
from sksurv.util import Surv
from utils import Logger
from itertools import product
# %%

def load_r_packages():
    ro.r('''  
    rm(list=ls())
    # library("scater")
    library("splatter")
    library(scran)
    library(Seurat)
    # library(preprocessCore)
    library(pROC)
    # library(APML2)
    # library(APML1)
    # library(APML0)

    library(ggplot2)
    library(dplyr)
    library(caret)
    set.seed(1)
    ''')


ro.r('.libPaths()')
load_r_packages()



    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

In [2]:
def detect_subpopulations(adata, surv, adata_new,   
                          entropy_threshold=0.7, 
                          num_heads=8,
                          feature_flavor='AE', 
                          feature_key=None, 
                          batch_key=None, 
                          use_covariates=False
                          ):
    
    adata = adata.copy()
    if use_covariates:
        covariates = surv[['group']].copy()
        surv = surv[['time', 'status']].copy()
    else:
        covariates = None

    if feature_key == 'X_nmf':
        assert feature_flavor == 'Custom'
        print('NMF feature extraction...')
        nmf = NMF(n_components=50, init='nndsvd', random_state=42, max_iter=500)
        adata.obsm[feature_key] = nmf.fit_transform(adata.X)
        adata_new.obsm[feature_key] = nmf.transform(adata_new[:, adata.var_names].X)
    elif feature_key == 'X_scaled':
        assert feature_flavor == 'Custom'
        print('Standard scaling feature extraction...')
        scaler = StandardScaler()
        adata.obsm[feature_key] = scaler.fit_transform(adata.X)
        adata_new.obsm[feature_key] = scaler.transform(adata_new[:, adata.var_names].X)
    else:
        assert feature_key is None

    adata, surv, model = scSurvivalRun(adata, 
        sample_column='sample',
        surv=surv,
        batch_key=batch_key,
        covariates=covariates,
        feature_flavor=feature_flavor,
        feature_key=feature_key,
        entropy_threshold=entropy_threshold,
        num_heads=num_heads,
        lambdas=(0.01, 1.0),
        pretrain_epochs=200,
        epochs=500,
        weight_decay=0.01,
        lr=0.001,
        patience=100,
        rec_likelihood='ZIG',
        do_scale_ae=False,
        beta=0.1, tau=0.2, 
        sample_size_ae=None,
        finetue_lr_factor=0.1,
        gene_weight_alpha=0.2,
        gamma_beta_weight=(0.1, 0.0),
        once_load_to_gpu=True,
        use_amp=False,
        fitnetune_strategy='alternating', # jointly, alternating, alternating_lightly,
        )

    data = adata.obs['attention'].values.reshape(-1, 1)
    kmeans = KMeans(n_clusters=2, random_state=42)
    kmeans.fit(data)
    cluster_centers = kmeans.cluster_centers_
    atten_thr = cluster_centers.flatten().mean()
    
    adata_new, _ = PredictIndSample(adata_new, adata, model)

    attention = adata_new.obs['attention'].values
    hazard_adj = adata_new.obs['hazard_adj'].values
    hazard = adata_new.obs['hazard'].values

    risk_group = np.array(['inattentive'] * attention.shape[0], dtype=object)
    risk_group[np.logical_and(attention >= atten_thr, hazard_adj > 0)] = 'higher'
    risk_group[np.logical_and(attention >= atten_thr, hazard_adj <= 0)] = 'lower'

    # higher -> bad.survival, lower -> good.survival, inattentive -> other 

    risk_group_recoded = np.array(['other'] * attention.shape[0], dtype=object)
    risk_group_recoded[risk_group == 'higher'] = 'bad.survival'
    risk_group_recoded[risk_group == 'lower'] = 'good.survival'

    clf_report = classification_report(adata_new.obs['sim_group'].values, risk_group_recoded, output_dict=True, zero_division=0)

    clf_report_df = pd.DataFrame(clf_report).T
    return clf_report_df, adata_new

def cross_validation_samples(adata, surv, 
                             entropy_threshold=0.7,
                             num_heads=8, 
                             feature_flavor='AE', 
                             feature_key=None, 
                             batch_key=None, 
                             use_covariates=False):
    # 交叉验证样本
    adata = adata.raw.to_adata()
    adata.obs['patient_no'] = adata.obs['sample']
    patients = adata.obs['patient_no'].unique()

    # K fold cross validation
    cv_hazards_adj_cells = np.zeros((adata.shape[0], ))
    surv['cv_hazards_adj_patient'] = 0.0
    surv['cv_hazard_percentile_patient'] = 0.0
    cindexs = []
    surv_test_all_folds = []

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    for i, (train_index, test_index) in enumerate(kf.split(patients)):

        print(f'fold {i}, train_size: {train_index.shape[0]}, test_size: {test_index.shape[0]}')
        train_patients = patients[train_index]
        test_patients = patients[test_index]

        # train
        adata_train = adata[adata.obs['patient_no'].isin(train_patients), :].copy()
    
        ## select HVGs on training set only
        sc.pp.highly_variable_genes(adata_train, n_top_genes=2000, subset=False, flavor='seurat')
        hvgs = adata_train.var[adata_train.var['highly_variable']].index.tolist() 
        adata_train = adata_train[:, hvgs]

        adata_test = adata[adata.obs['patient_no'].isin(test_patients), :].copy()
        adata_test = adata_test[:, hvgs]

        if feature_key == 'X_nmf':
            assert feature_flavor == 'Custom'
            print('NMF feature extraction...')
            nmf = NMF(n_components=50, init='nndsvd', random_state=42, max_iter=500)
            adata_train.obsm[feature_key] = nmf.fit_transform(adata_train.X)
            adata_test.obsm[feature_key] = nmf.transform(adata_test.X)
        elif feature_key == 'X_scaled':
            assert feature_flavor == 'Custom'
            print('Standard scaling feature extraction...') 
            scaler = StandardScaler()
            adata_train.obsm[feature_key] = scaler.fit_transform(adata_train.X)
            adata_test.obsm[feature_key] = scaler.transform(adata_test.X)
        else:
            assert feature_key is None


        surv_train = surv.loc[surv.index.isin(train_patients), :].copy()
        if use_covariates:
            covariates_train = surv_train[['group']].copy()
        else:
            covariates_train = None

        adata_train, surv_train, model = scSurvivalRun(
            adata_train,
            sample_column='sample',
            surv=surv_train,
            batch_key=batch_key,
            covariates=covariates_train,
            feature_flavor=feature_flavor,
            feature_key=feature_key,
            entropy_threshold=entropy_threshold,
            num_heads=num_heads,
            validate=True,
            validate_ratio=0.2,
            validate_metric='ccindex',
            lambdas=(0.01, 1.0),
            pretrain_epochs=200,
            epochs=500,
            weight_decay=0.01,
            lr=0.001,
            patience=100,
            rec_likelihood='ZIG',
            do_scale_ae=False,
            beta=0.1, tau=0.2, 
            sample_size_ae=None,
            finetue_lr_factor=0.1,
            gene_weight_alpha=0.2,
            gamma_beta_weight=(0.1, 0.0),
            once_load_to_gpu=True,
            use_amp=False,
            fitnetune_strategy='alternating', # jointly, alternating, alternating_lightly,
            )  
        
        
        train_cindex = concordance_index(surv_train['time'], -surv_train['patient_hazards'], surv_train['status'])
        print(f'train c-index: {train_cindex:.4f}')

        # test
        print('testing...')
        

        with contextlib.redirect_stdout(f):
            for test_patient in test_patients:
                adata_test_patient = adata_test[adata_test.obs['patient_no'] == test_patient, :].copy()
                if use_covariates:
                    covariates_test_patient = surv.loc[[test_patient], ['group']].copy()
                    adata_test_patient, patient_hazard = PredictIndSample(adata_test_patient, adata_train, model, covariates_new=covariates_test_patient)
                else:
                    adata_test_patient, patient_hazard = PredictIndSample(adata_test_patient, adata_train, model)
                    
                cv_hazards_adj_cells[adata.obs['patient_no'] == test_patient] = adata_test_patient.obs['hazard_adj'].values
                surv.loc[surv.index == test_patient, 'cv_hazards_adj_patient'] = patient_hazard
                surv.loc[surv.index == test_patient, 'cv_hazard_percentile_patient'] = percentileofscore(surv_train['patient_hazards'], patient_hazard, kind='rank')

        surv_test = surv.loc[surv.index.isin(test_patients), :]
        c_index = concordance_index(surv_test['time'], -surv_test['cv_hazards_adj_patient'], surv_test['status'])

        cindexs.append(c_index)
        surv_test_all_folds.append(surv_test)

        print(f'c-index: {c_index:.4f}')
        print('='*50)

        # if i == 0:
        #     break

    mean_cindex = np.mean(cindexs)
    std_cindex = np.std(cindexs)

    print(f'mean c-index: {mean_cindex:.4f} ± {std_cindex:.4f}')
    cindexs_df = pd.DataFrame(cindexs, columns=['c-index'], index=['fold%d' % i for i in range(5)])

    cindex_results = {
        'mean_cindex': mean_cindex,
        'std_cindex': std_cindex,
        'cindexs_df': cindexs_df
    }

    return cindex_results


def celltype_proportion_coxph(adata, surv, use_covariates=False):

    if use_covariates:
        covariates = surv[['group']].copy()
        surv = surv[['time', 'status']].copy()

    adata = adata.copy()
    print('Clustering cells...')
    sc.pp.scale(adata, max_value=10)
    sc.pp.pca(adata, n_comps=50)
    sc.pp.neighbors(adata, n_neighbors=15, n_pcs=50)
    sc.tl.leiden(adata)

    meta = adata.obs.copy()
    cell_fraction = meta[['sample', 'leiden']].value_counts(normalize=False).unstack(fill_value=0)
    cell_fraction = cell_fraction.div(cell_fraction.sum(axis=1), axis=0)
    surv = surv.loc[cell_fraction.index, :].copy()
    
    surv['time'] = surv['time'].astype(float)
    surv['event'] = surv['status'].astype(int)

    if use_covariates:
        covariates = covariates.loc[cell_fraction.index, :].copy()
        covariates = pd.get_dummies(covariates, drop_first=True).astype(float)
        X = pd.concat([cell_fraction, covariates], axis=1).values
    else:
        X = cell_fraction.values
    y = Surv.from_dataframe('event', 'time', surv)

    print('Fitting Cox model with cell type proportions...')
    cindex_results = nested_cv_coxnet(
        X, y,
        outer_mode='kfold',
        outer_splits=5,           # Used when outer_mode='kfold'
        inner_splits=5,
        # alpha_grid=np.logspace(-2, 1, 50),
        # alpha_grid=[0.1],
        n_alphas=50,
        alpha_min_ratio=0.01,
        l1_ratio=0.1,
        random_state=42,
        n_jobs=-1,
        return_coef=True
    )
    return cindex_results

def pseudo_bulk_coxph(adata, surv, use_covariates=False):
    if use_covariates:
        covariates = surv[['group']].copy()
        surv = surv[['time', 'status']].copy()
    
    adata = adata.raw.to_adata()

    print('Calculating pseudo-bulk expression...')
    patients = adata.obs['sample'].unique().tolist()
    pseudo_bulk_list = []
    for patient in tqdm(patients):
        adata_sub = adata[adata.obs['sample'] == patient, :].copy()
        mean_expr = np.asarray(adata_sub.X.mean(axis=0)).flatten()
        pseudo_bulk_list.append(mean_expr)

    pseudo_bulk = pd.DataFrame(data=np.vstack(pseudo_bulk_list),
                                index=patients,
                                columns=adata.var_names.tolist())
    
    surv = surv.loc[pseudo_bulk.index, :].copy()
    surv['time'] = surv['time'].astype(float)
    surv['event'] = surv['status'].astype(int)

    if use_covariates:
        covariates = covariates.loc[pseudo_bulk.index, :].copy()
        covariates = pd.get_dummies(covariates, drop_first=True).astype(float)
        X = pd.concat([pseudo_bulk, covariates], axis=1).values
    else:
        X = pseudo_bulk.values
    y = Surv.from_dataframe('event', 'time', surv)

    print('Fitting Cox model with pseudo-bulk expression...')
    cindex_results = nested_cv_coxnet(
        X, y,
        outer_mode='kfold',
        outer_splits=5,           # Used when outer_mode='kfold'
        inner_splits=5,
        # alpha_grid=np.logspace(-1, 2, 50),
        # alpha_grid=[0],
        n_alphas=50,
        alpha_min_ratio=0.01,
        l1_ratio=1.0,
        random_state=42,
        n_jobs=-1,
        return_coef=True,
        select_features=True,
        n_top=2000,
    )

    return cindex_results

In [3]:
def log_clf_report_df(logger, clf_report_df=None):
    if clf_report_df is None:
        clf_rst = {
            'precision': 'NA',
            'recall': 'NA',
            'f1-score': 'NA',
        }
        for cls in ['good.survival', 'bad.survival', 'other']:
            for metric in ['precision', 'recall', 'f1-score']:
                key = f'{cls}_{metric}'
                clf_rst[key] = 'NA'
    else:
        clf_rst = {
            'precision': clf_report_df.loc['macro avg', 'precision'],
            'recall': clf_report_df.loc['macro avg', 'recall'],
            'f1-score': clf_report_df.loc['macro avg', 'f1-score'],
        }

        for cls in ['good.survival', 'bad.survival', 'other']:
            for metric in ['precision', 'recall', 'f1-score']:
                key = f'{cls}_{metric}'
                if cls in clf_report_df.index:
                    clf_rst[key] = clf_report_df.loc[cls, metric]
                else:
                    clf_rst[key] = 0.0
    logger.log_dict(clf_rst)
    return logger

def log_cindex_results(logger, cindex_results):
    logger.log('mean_cindex', cindex_results['mean_cindex'])
    logger.log('std_cindex', cindex_results['std_cindex'])
    return logger

# alg params

def get_alg_params_on_methods(method):
    if method == 'scSurvival(VAE)':
        return {
            'feature_flavor': 'AE', 
            'feature_key': None,
            'num_heads': 8,
        }
    elif method == 'scSurvival(PCA)':
        return {
            'feature_flavor': 'PCA', 
            'feature_key': None,
            'num_heads': 5,
        }
    elif method == 'scSurvival(NMF)':
        return {
            'feature_flavor': 'Custom', 
            'feature_key': 'X_nmf',
            'num_heads': 5,
        }
    elif method == 'scSurvival(HVG2000)':
        return {
            'feature_flavor': 'Custom', 
            'feature_key': 'X_scaled',
            'num_heads': 8,
        }
    elif method == 'celltype_proportion_coxph':
        return {}
    elif method == 'pseudo_bulk_coxph':
        return {}
    else:
        raise ValueError(f'Unknown method: {method}')

def get_alg_params_on_sim(sim):
    if sim == 'sim1':
        return {
            'entropy_threshold': 0.7,
            'batch_key': None,
            'use_covariates': False
        }
    elif sim == 'sim2':
        return {
            'entropy_threshold': 0.7,
            'batch_key': None, # for pca, nmf, scaled, set to None
            'use_covariates': False
        }
    elif sim == 'sim3':
        return {
            'entropy_threshold': 0.7,
            'batch_key': None,
            'use_covariates': False
        }
    elif sim == 'sim4':
        return {
            'entropy_threshold': 0.7,
            'batch_key': None,
            'use_covariates': False
        }
    elif sim == 'sim5':
        return {
            'entropy_threshold': 0.7,
            'batch_key': None,
            'use_covariates': True
        }
    elif sim == 'sim6':
        return {
            'entropy_threshold': 0.7,
            'batch_key': None,
            'use_covariates': False
        }
    else:
        raise ValueError(f'Unknown sim: {sim}')
    
def load_scSurvival_vae_results(rst_type='clf_report', sim_case='sim1', seed=1, **params):
    '''
    rst_type: 'clf_report' or 'cindex_results'
    '''

    result_root_path = f'./results/revision-{sim_case}-python/'

    if rst_type == 'clf_report':
        file_path = f'{result_root_path}/cell_subpopulation_logs.csv'
        df = pd.read_csv(file_path)

    elif rst_type == 'cindex_results':
        file_path = f'{result_root_path}/cv_logs.csv'
        df = pd.read_csv(file_path)

    rst_check = df['seed'] == seed
    for key, value in params.items():
        rst_check = rst_check & (df[key] == value)
    
    df_sub = df.loc[rst_check, :]
    assert df_sub.shape[0] == 1, f'Expected 1 result, but got {df_sub.shape[0]}'

    df_sub_dict = df_sub.to_dict(orient='records')[0]

    # remove seed and params from dict
    df_sub_dict.pop('seed', None)
    for key in params.keys():
        df_sub_dict.pop(key, None)

    return df_sub_dict

In [4]:
represent_cases = {
    'sim1': {
        'cell_surv_ratio': [0.15, 0.03]
    },
    'sim2': {
        'ds_rate': [0.2]
    },
    'sim3': {
        'censor_prob': [0.7],
        'tr': [2]
    },
    'sim4': {
        'num_bg_clusters': [5]
    },
    'sim5': {
        'group_effect': [3],
        'high_group_prob': [0.5]
    },
    'sim6': {
        'sub_cluster_ratio': [0.3],
        'de_facloc': [0.35]
    }
}
seeds = range(1, 11)
methods = [
    'scSurvival(VAE)', 
    'scSurvival(PCA)',
    'scSurvival(NMF)',
    'scSurvival(HVG2000)',
    'celltype_proportion_coxph', 
    'pseudo_bulk_coxph',
]

# methods = [
#     'scSurvival(HVG2000)',
# ]

In [ ]:
save_root_path = './benchmark_results/'
if not os.path.exists(save_root_path):
    os.makedirs(save_root_path)

logger = Logger(f'{save_root_path}/log_cindexes_hvg2000.csv')
for seed in [1]:
    for i, (case, params) in enumerate(represent_cases.items()):
        sim = importlib.import_module(f'simu_type_{i+1}')
        
        keys, values = zip(*params.items())
        combos = [dict(zip(keys, v)) for v in product(*values)]
        for param in combos:
            print(f'Running {case} with params: {param} and seed: {seed}')
            # param['seed'] = seed
            adata, surv, adata_new = sim.create_data(seed=seed, **param)

            for method in methods:
                print(f'Running method: {method}')
                logger.log('seed', seed)
                logger.log('case', case)
                logger.log('method', method)
                logger.log('params', str(param))

                if method == 'scSurvival(VAE)':
                    cindex_results = load_scSurvival_vae_results(rst_type='cindex_results', sim_case=case, seed=seed, **param)
                    logger.log_dict(cindex_results)
                    logger.get_logs_df()
                    continue

                alg_params_on_methods = get_alg_params_on_methods(method)
                alg_params_on_sim = get_alg_params_on_sim(case)
                alg_params = {**alg_params_on_methods, **alg_params_on_sim}

                if 'cell_surv_ratio' in param:
                    if param['cell_surv_ratio'] == 0.03:
                        alg_params['entropy_threshold'] = 0.5
                
                if method.startswith('scSurvival'):
                    cindex_results = cross_validation_samples(adata, surv, **alg_params)
                    logger = log_cindex_results(logger, cindex_results)
                elif method == 'celltype_proportion_coxph':
                    use_covariates = alg_params.get('use_covariates', False)
                    cindex_results = celltype_proportion_coxph(adata, surv, use_covariates=use_covariates)
                    logger = log_cindex_results(logger, cindex_results)
                elif method == 'pseudo_bulk_coxph':
                    use_covariates = alg_params.get('use_covariates', False)
                    cindex_results = pseudo_bulk_coxph(adata, surv, use_covariates=use_covariates)
                    logger = log_cindex_results(logger, cindex_results)
                
                logger.get_logs_df()
            # raise ValueError('stop here for testing')

In [5]:
save_root_path = './benchmark_results/'
if not os.path.exists(save_root_path):
    os.makedirs(save_root_path)

logger = Logger(f'{save_root_path}/log_detecting_cells.csv')
for seed in seeds:
    for i, (case, params) in enumerate(represent_cases.items()):
        sim = importlib.import_module(f'simu_type_{i+1}')
        
        keys, values = zip(*params.items())
        combos = [dict(zip(keys, v)) for v in product(*values)]
        for param in combos:
            print(f'Running {case} with params: {param} and seed: {seed}')
            # param['seed'] = seed
            adata, surv, adata_new = sim.create_data(seed=seed, **param)

            for method in methods:
                

                if not method.startswith('scSurvival'):
                    continue

                print(f'Running method: {method}')
                
                logger.log('seed', seed)
                logger.log('case', case)
                logger.log('params', str(param))
                logger.log('method', method)

                if method == 'scSurvival(VAE)':
                    clf_report_dict = load_scSurvival_vae_results(rst_type='clf_report', sim_case=case, seed=seed, **param)
                    logger.log_dict(clf_report_dict)
                    logger.get_logs_df()
                    continue

                alg_params_on_methods = get_alg_params_on_methods(method)
                alg_params_on_sim = get_alg_params_on_sim(case)
                alg_params = {**alg_params_on_methods, **alg_params_on_sim}

                if 'cell_surv_ratio' in param:
                    if param['cell_surv_ratio'] == 0.03:
                        alg_params['entropy_threshold'] = 0.5

                clf_report_df, adata_new = detect_subpopulations(adata, surv, adata_new, **alg_params)
                logger = log_clf_report_df(logger, clf_report_df)
                
                logger.get_logs_df()

                # raise ValueError('stop here for testing')

Running sim1 with params: {'cell_surv_ratio': 0.15} and seed: 1
  |======================================================================| 100%

100%|██████████| 100/100 [00:00<00:00, 48121.89it/s]


Running method: scSurvival(VAE)
Running method: scSurvival(PCA)
Run PCA...


Training:  64%|██████▍   | 319/500 [01:31<00:51,  3.50it/s, atten_entropy=0.813, cox_loss=2.26, loss=2.38]


Added hazard and attention to adata.obs.
Added patient_hazards to surv.
gene missing rate: 0.00%
Added hazard and attention to adata_new.obs.
Running method: scSurvival(NMF)
NMF feature extraction...


...

Added hazard and attention to adata.obs.
Added patient_hazards to surv.
gene missing rate: 0.00%
Added hazard and attention to adata_new.obs.
Running method: scSurvival(NMF)
NMF feature extraction...


Training: 100%|██████████| 500/500 [02:32<00:00,  3.29it/s, atten_entropy=0.689, cox_loss=2.77, loss=2.77]


Added hazard and attention to adata.obs.
Added patient_hazards to surv.
gene missing rate: 0.00%
Added hazard and attention to adata_new.obs.
Running method: scSurvival(HVG2000)
Standard scaling feature extraction...


Training:  20%|██        | 101/500 [01:01<04:03,  1.64it/s, atten_entropy=0.652, cox_loss=3.59, loss=3.59]


Added hazard and attention to adata.obs.
Added patient_hazards to surv.
gene missing rate: 0.00%
Added hazard and attention to adata_new.obs.


In [10]:
adata

AnnData object with n_obs × n_vars = 100000 × 2000
    obs: 'sample', 'hazard', 'attention', 'hazard_adj'
    var: 'mean', 'std'
    uns: 'pca'
    obsm: 'X_pca'
    varm: 'PCs'